# CORRECTED EMOTION DETECTION TRAINING
## Using j-hartmann/emotion-english-distilroberta-base with Verification

**CRITICAL**: This notebook ensures we use the correct specialized emotion model
and verifies it's working properly before training.

**Target**: Reliable 75-85% F1 score with proper emotion-specialized model

In [17]:
!git clone https://github.com/uelkerd/SAMO--DL.git
!pwd

Cloning into 'SAMO--DL'...
remote: Enumerating objects: 2901, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 2901 (delta 140), reused 163 (delta 74), pack-reused 2647 (from 1)
Receiving objects: 100% (2901/2901), 24.28 MiB | 15.20 MiB/s, done.
Resolving deltas: 100% (2023/2023), done.
/content/SAMO--DL


In [16]:
%cd SAMO--DL
# Install required packages
!pip install transformers datasets torch scikit-learn numpy pandas huggingface_hub

/content/SAMO--DL
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninsta

In [18]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import json
import warnings
warnings.filterwarnings('ignore')

print('✅ Packages imported successfully')

✅ Packages imported successfully


In [19]:
# CRITICAL: Verify we can access the specialized model
print('🔍 VERIFYING SPECIALIZED MODEL ACCESS')
print('=' * 50)

specialized_model_name = 'j-hartmann/emotion-english-distilroberta-base'

try:
    print(f'Testing access to: {specialized_model_name}')
    test_tokenizer = AutoTokenizer.from_pretrained(specialized_model_name)
    test_model = AutoModelForSequenceClassification.from_pretrained(specialized_model_name)

    print('✅ SUCCESS: Specialized model loaded!')
    print(f'Model type: {test_model.config.model_type}')
    print(f'Architecture: {test_model.config.architectures[0]}')
    print(f'Hidden layers: {test_model.config.num_hidden_layers}')
    print(f'Hidden size: {test_model.config.hidden_size}')
    print(f'Number of labels: {test_model.config.num_labels}')
    print(f'Original labels: {test_model.config.id2label}')

    # Verify it's actually DistilRoBERTa
    if test_model.config.num_hidden_layers == 6 and 'distil' in test_model.config.model_type.lower():
        print('✅ CONFIRMED: This is DistilRoBERTa architecture')
    else:
        print('⚠️  WARNING: This may not be the expected DistilRoBERTa model')

except Exception as e:
    print(f'❌ ERROR: Cannot access specialized model: {str(e)}')
    print('\n🔧 FALLBACK: Using roberta-base instead')
    specialized_model_name = 'roberta-base'
    test_tokenizer = AutoTokenizer.from_pretrained(specialized_model_name)
    test_model = AutoModelForSequenceClassification.from_pretrained(specialized_model_name, num_labels=12)
    print(f'✅ Fallback model loaded: {specialized_model_name}')

🔍 VERIFYING SPECIALIZED MODEL ACCESS
Testing access to: j-hartmann/emotion-english-distilroberta-base
✅ SUCCESS: Specialized model loaded!
Model type: roberta
Architecture: RobertaForSequenceClassification
Hidden layers: 6
Hidden size: 768
Number of labels: 7
Original labels: {0: 'anger', 1: 'disgust', 2: 'fear', 3: 'joy', 4: 'neutral', 5: 'sadness', 6: 'surprise'}
⚠️  WARNING: This may not be the expected DistilRoBERTa model


In [20]:
# Define our emotion classes
emotions = ['anxious', 'calm', 'content', 'excited', 'frustrated', 'grateful', 'happy', 'hopeful', 'overwhelmed', 'proud', 'sad', 'tired']
print(f'🎯 Our emotion classes: {emotions}')
print(f'📊 Number of emotions: {len(emotions)}')

🎯 Our emotion classes: ['anxious', 'calm', 'content', 'excited', 'frustrated', 'grateful', 'happy', 'hopeful', 'overwhelmed', 'proud', 'sad', 'tired']
📊 Number of emotions: 12


In [21]:
# Create balanced training dataset
print('📊 CREATING BALANCED DATASET')
print('=' * 40)

balanced_data = [
    # anxious (12 samples)
    {'text': 'I feel anxious about the presentation.', 'label': 0},
    {'text': 'I am anxious about the future.', 'label': 0},
    {'text': 'This makes me feel anxious.', 'label': 0},
    {'text': 'I am feeling anxious today.', 'label': 0},
    {'text': 'The uncertainty makes me anxious.', 'label': 0},
    {'text': 'I feel anxious about the results.', 'label': 0},
    {'text': 'This situation is making me anxious.', 'label': 0},
    {'text': 'I am anxious about the meeting.', 'label': 0},
    {'text': 'The pressure is making me anxious.', 'label': 0},
    {'text': 'I feel anxious about the decision.', 'label': 0},
    {'text': 'This is causing me anxiety.', 'label': 0},
    {'text': 'I am anxious about the changes.', 'label': 0},

    # calm (12 samples)
    {'text': 'I feel calm and peaceful.', 'label': 1},
    {'text': 'I am feeling calm today.', 'label': 1},
    {'text': 'This makes me feel calm.', 'label': 1},
    {'text': 'I am calm about the situation.', 'label': 1},
    {'text': 'I feel calm and relaxed.', 'label': 1},
    {'text': 'This gives me a sense of calm.', 'label': 1},
    {'text': 'I am feeling calm and centered.', 'label': 1},
    {'text': 'This brings me calm.', 'label': 1},
    {'text': 'I feel calm and at peace.', 'label': 1},
    {'text': 'I am calm about the outcome.', 'label': 1},
    {'text': 'This creates a feeling of calm.', 'label': 1},
    {'text': 'I feel calm and collected.', 'label': 1},

    # content (12 samples)
    {'text': 'I feel content with my life.', 'label': 2},
    {'text': 'I am content with the results.', 'label': 2},
    {'text': 'This makes me feel content.', 'label': 2},
    {'text': 'I am feeling content today.', 'label': 2},
    {'text': 'I feel content and satisfied.', 'label': 2},
    {'text': 'This gives me contentment.', 'label': 2},
    {'text': 'I am content with my choices.', 'label': 2},
    {'text': 'I feel content and fulfilled.', 'label': 2},
    {'text': 'This brings me contentment.', 'label': 2},
    {'text': 'I am content with the situation.', 'label': 2},
    {'text': 'I feel content and at ease.', 'label': 2},
    {'text': 'This creates contentment in me.', 'label': 2},

    # excited (12 samples)
    {'text': 'I am excited about the new opportunity.', 'label': 3},
    {'text': 'I feel excited about the future.', 'label': 3},
    {'text': 'This makes me feel excited.', 'label': 3},
    {'text': 'I am feeling excited today.', 'label': 3},
    {'text': 'I feel excited and enthusiastic.', 'label': 3},
    {'text': 'This gives me excitement.', 'label': 3},
    {'text': 'I am excited about the project.', 'label': 3},
    {'text': 'I feel excited and motivated.', 'label': 3},
    {'text': 'This brings me excitement.', 'label': 3},
    {'text': 'I am excited about the possibilities.', 'label': 3},
    {'text': 'I feel excited and energized.', 'label': 3},
    {'text': 'This creates excitement in me.', 'label': 3},

    # frustrated (12 samples)
    {'text': 'I am so frustrated with this project.', 'label': 4},
    {'text': 'I feel frustrated about the situation.', 'label': 4},
    {'text': 'This makes me feel frustrated.', 'label': 4},
    {'text': 'I am feeling frustrated today.', 'label': 4},
    {'text': 'I feel frustrated and annoyed.', 'label': 4},
    {'text': 'This gives me frustration.', 'label': 4},
    {'text': 'I am frustrated with the results.', 'label': 4},
    {'text': 'I feel frustrated and irritated.', 'label': 4},
    {'text': 'This brings me frustration.', 'label': 4},
    {'text': 'I am frustrated with the process.', 'label': 4},
    {'text': 'I feel frustrated and upset.', 'label': 4},
    {'text': 'This creates frustration in me.', 'label': 4},

    # grateful (12 samples)
    {'text': 'I am grateful for all the support.', 'label': 5},
    {'text': 'I feel grateful for the opportunity.', 'label': 5},
    {'text': 'This makes me feel grateful.', 'label': 5},
    {'text': 'I am feeling grateful today.', 'label': 5},
    {'text': 'I feel grateful and thankful.', 'label': 5},
    {'text': 'This gives me gratitude.', 'label': 5},
    {'text': 'I am grateful for the help.', 'label': 5},
    {'text': 'I feel grateful and appreciative.', 'label': 5},
    {'text': 'This brings me gratitude.', 'label': 5},
    {'text': 'I am grateful for the kindness.', 'label': 5},
    {'text': 'I feel grateful and blessed.', 'label': 5},
    {'text': 'This creates gratitude in me.', 'label': 5},

    # happy (12 samples)
    {'text': 'I am feeling really happy today!', 'label': 6},
    {'text': 'I feel happy about the news.', 'label': 6},
    {'text': 'This makes me feel happy.', 'label': 6},
    {'text': 'I am feeling happy today.', 'label': 6},
    {'text': 'I feel happy and joyful.', 'label': 6},
    {'text': 'This gives me happiness.', 'label': 6},
    {'text': 'I am happy with the results.', 'label': 6},
    {'text': 'I feel happy and delighted.', 'label': 6},
    {'text': 'This brings me happiness.', 'label': 6},
    {'text': 'I am happy about the success.', 'label': 6},
    {'text': 'I feel happy and cheerful.', 'label': 6},
    {'text': 'This creates happiness in me.', 'label': 6},

    # hopeful (12 samples)
    {'text': 'I am hopeful for the future.', 'label': 7},
    {'text': 'I feel hopeful about the outcome.', 'label': 7},
    {'text': 'This makes me feel hopeful.', 'label': 7},
    {'text': 'I am feeling hopeful today.', 'label': 7},
    {'text': 'I feel hopeful and optimistic.', 'label': 7},
    {'text': 'This gives me hope.', 'label': 7},
    {'text': 'I am hopeful about the changes.', 'label': 7},
    {'text': 'I feel hopeful and positive.', 'label': 7},
    {'text': 'This brings me hope.', 'label': 7},
    {'text': 'I am hopeful about the possibilities.', 'label': 7},
    {'text': 'I feel hopeful and confident.', 'label': 7},
    {'text': 'This creates hope in me.', 'label': 7},

    # overwhelmed (12 samples)
    {'text': 'I am feeling overwhelmed with tasks.', 'label': 8},
    {'text': 'I feel overwhelmed by the workload.', 'label': 8},
    {'text': 'This makes me feel overwhelmed.', 'label': 8},
    {'text': 'I am feeling overwhelmed today.', 'label': 8},
    {'text': 'I feel overwhelmed and stressed.', 'label': 8},
    {'text': 'This gives me overwhelm.', 'label': 8},
    {'text': 'I am overwhelmed by the situation.', 'label': 8},
    {'text': 'I feel overwhelmed and exhausted.', 'label': 8},
    {'text': 'This brings me overwhelm.', 'label': 8},
    {'text': 'I am overwhelmed by the pressure.', 'label': 8},
    {'text': 'I feel overwhelmed and drained.', 'label': 8},
    {'text': 'This creates overwhelm in me.', 'label': 8},

    # proud (12 samples)
    {'text': 'I am proud of my accomplishments.', 'label': 9},
    {'text': 'I feel proud of the results.', 'label': 9},
    {'text': 'This makes me feel proud.', 'label': 9},
    {'text': 'I am feeling proud today.', 'label': 9},
    {'text': 'I feel proud and accomplished.', 'label': 9},
    {'text': 'This gives me pride.', 'label': 9},
    {'text': 'I am proud of the achievement.', 'label': 9},
    {'text': 'I feel proud and satisfied.', 'label': 9},
    {'text': 'This brings me pride.', 'label': 9},
    {'text': 'I am proud of the success.', 'label': 9},
    {'text': 'I feel proud and confident.', 'label': 9},
    {'text': 'This creates pride in me.', 'label': 9},

    # sad (12 samples)
    {'text': 'I feel sad about the loss.', 'label': 10},
    {'text': 'I am sad about the situation.', 'label': 10},
    {'text': 'This makes me feel sad.', 'label': 10},
    {'text': 'I am feeling sad today.', 'label': 10},
    {'text': 'I feel sad and down.', 'label': 10},
    {'text': 'This gives me sadness.', 'label': 10},
    {'text': 'I am sad about the outcome.', 'label': 10},
    {'text': 'I feel sad and depressed.', 'label': 10},
    {'text': 'This brings me sadness.', 'label': 10},
    {'text': 'I am sad about the news.', 'label': 10},
    {'text': 'I feel sad and heartbroken.', 'label': 10},
    {'text': 'This creates sadness in me.', 'label': 10},

    # tired (12 samples)
    {'text': 'I am tired from working all day.', 'label': 11},
    {'text': 'I feel tired of the routine.', 'label': 11},
    {'text': 'This makes me feel tired.', 'label': 11},
    {'text': 'I am feeling tired today.', 'label': 11},
    {'text': 'I feel tired and exhausted.', 'label': 11},
    {'text': 'This gives me tiredness.', 'label': 11},
    {'text': 'I am tired of the situation.', 'label': 11},
    {'text': 'I feel tired and worn out.', 'label': 11},
    {'text': 'This brings me tiredness.', 'label': 11},
    {'text': 'I am tired of the stress.', 'label': 11},
    {'text': 'I feel tired and fatigued.', 'label': 11},
    {'text': 'This creates tiredness in me.', 'label': 11}
]

print(f'✅ Created balanced dataset with {len(balanced_data)} samples')
print(f'📊 Samples per emotion: {len(balanced_data) // len(emotions)}')

# Verify balance
emotion_counts = {}
for item in balanced_data:
    emotion = emotions[item['label']]
    emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1

print('\n📈 Emotion distribution:')
for emotion, count in emotion_counts.items():
    print(f'  {emotion}: {count} samples')

📊 CREATING BALANCED DATASET
✅ Created balanced dataset with 144 samples
📊 Samples per emotion: 12

📈 Emotion distribution:
  anxious: 12 samples
  calm: 12 samples
  content: 12 samples
  excited: 12 samples
  frustrated: 12 samples
  grateful: 12 samples
  happy: 12 samples
  hopeful: 12 samples
  overwhelmed: 12 samples
  proud: 12 samples
  sad: 12 samples
  tired: 12 samples


In [22]:
# Split data with proper validation
print('🔀 SPLITTING DATA WITH VALIDATION')
print('=' * 40)

train_data, val_data = train_test_split(balanced_data, test_size=0.2, random_state=42, stratify=[d['label'] for d in balanced_data])

print(f'Training samples: {len(train_data)}')
print(f'Validation samples: {len(val_data)}')

# Convert to datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

print('✅ Datasets created successfully')

🔀 SPLITTING DATA WITH VALIDATION
Training samples: 115
Validation samples: 29
✅ Datasets created successfully


In [23]:
# Load the CORRECT specialized model
print('🔧 LOADING SPECIALIZED MODEL')
print('=' * 40)

tokenizer = AutoTokenizer.from_pretrained(specialized_model_name)

# For specialized model, we need to resize the classifier for our 12 emotions
model = AutoModelForSequenceClassification.from_pretrained(
    specialized_model_name,
    num_labels=12,
    ignore_mismatched_sizes=True  # This is the key to resizing the classifier
)

print('✅ Loaded specialized emotion model and resized for 12 emotions')


# Update model config with our emotion labels
model.config.id2label = {i: emotion for i, emotion in enumerate(emotions)}
model.config.label2id = {emotion: i for i, emotion in enumerate(emotions)}

print(f'Model type: {model.config.model_type}')
print(f'Architecture: {model.config.architectures[0]}')
print(f'Hidden layers: {model.config.num_hidden_layers}')
print(f'Hidden size: {model.config.hidden_size}')
print(f'Number of labels: {model.config.num_labels}')
print(f'Our labels: {model.config.id2label}')

🔧 LOADING SPECIALIZED MODEL


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([12, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([12]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Loaded specialized emotion model and resized for 12 emotions
Model type: roberta
Architecture: RobertaForSequenceClassification
Hidden layers: 6
Hidden size: 768
Number of labels: 12
Our labels: {0: 'anxious', 1: 'calm', 2: 'content', 3: 'excited', 4: 'frustrated', 5: 'grateful', 6: 'happy', 7: 'hopeful', 8: 'overwhelmed', 9: 'proud', 10: 'sad', 11: 'tired'}


In [24]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

print('✅ Data tokenized successfully')

Map:   0%|          | 0/115 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

✅ Data tokenized successfully


In [25]:
# Training arguments with proper settings
print('⚙️  CONFIGURING TRAINING ARGUMENTS')
print('=' * 40)

training_args = TrainingArguments(
    output_dir='./corrected_emotion_model',
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Increased for A100
    per_device_eval_batch_size=16,   # Increased for A100
    num_train_epochs=5,
    weight_decay=0.01,  # Regularization
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    warmup_steps=100,
    dataloader_num_workers=0,
    save_total_limit=3  # Keep only best 3 checkpoints
)

print('✅ Training arguments configured')

⚙️  CONFIGURING TRAINING ARGUMENTS
✅ Training arguments configured


In [26]:
# Custom metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Calculate metrics
    report = classification_report(labels, predictions, target_names=emotions, output_dict=True)

    return {
        'f1': report['weighted avg']['f1-score'],
        'accuracy': report['accuracy'],
        'precision': report['weighted avg']['precision'],
        'recall': report['weighted avg']['recall']
    }

In [27]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print('✅ Trainer initialized successfully')

✅ Trainer initialized successfully


In [28]:
# Train the model
print('🚀 STARTING TRAINING')
print('=' * 40)
print(f'Using model: {specialized_model_name}')
print(f'Training samples: {len(train_data)}')
print(f'Validation samples: {len(val_data)}')
print('\nTraining...')

trainer.train()

print('✅ Training completed successfully')

🚀 STARTING TRAINING
Using model: j-hartmann/emotion-english-distilroberta-base
Training samples: 115
Validation samples: 29

Training...


Step,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall
10,2.507600,2.450205,0.128736,0.137931,0.137931,0.137931
20,2.444900,2.365234,0.179803,0.206897,0.213793,0.206897
30,2.341800,2.226004,0.346470,0.413793,0.312069,0.413793
40,2.203500,2.038876,0.754789,0.793103,0.755337,0.793103


✅ Training completed successfully


In [29]:
# Evaluate the model
print('📊 EVALUATING MODEL')
print('=' * 40)

results = trainer.evaluate()
print(f'Final F1 Score: {results["eval_f1"]:.3f}')
print(f'Final Accuracy: {results["eval_accuracy"]:.3f}')
print(f'Final Precision: {results["eval_precision"]:.3f}')
print(f'Final Recall: {results["eval_recall"]:.3f}')

📊 EVALUATING MODEL


Final F1 Score: 0.755
Final Accuracy: 0.793
Final Precision: 0.755
Final Recall: 0.793


In [30]:
# CRITICAL: Test on diverse examples to verify reliability
print('🧪 RELIABILITY TESTING')
print('=' * 40)

test_examples = [
    'I am feeling really happy today!',
    'I am so frustrated with this project.',
    'I feel anxious about the presentation.',
    'I am grateful for all the support.',
    'I am feeling overwhelmed with tasks.',
    'I am proud of my accomplishments.',
    'I feel sad about the loss.',
    'I am tired from working all day.',
    'I feel calm and peaceful.',
    'I am excited about the new opportunity.',
    'I feel content with my life.',
    'I am hopeful for the future.'
]

print('Testing on diverse examples...')
correct = 0
predictions_by_emotion = {emotion: 0 for emotion in emotions}

device = model.device  # Get the device the model is on

for text in test_examples:
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the correct device
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(predictions, dim=1).item()
        confidence = predictions[0][predicted_class].item()

    predicted_emotion = emotions[predicted_class]
    predictions_by_emotion[predicted_emotion] += 1

    expected_emotion = None
    for emotion in emotions:
        if emotion in text.lower():
            expected_emotion = emotion
            break

    if expected_emotion and predicted_emotion == expected_emotion:
        correct += 1
        status = '✅'
    else:
        status = '❌'

    print(f'{status} {text} → {predicted_emotion} (expected: {expected_emotion}, confidence: {confidence:.3f})')

accuracy = correct / len(test_examples)
print(f'\n📊 Test Accuracy: {accuracy:.1%}')

# Check for bias
print('\n🎯 Bias Analysis:')
for emotion, count in predictions_by_emotion.items():
    percentage = count / len(test_examples) * 100
    print(f'  {emotion}: {count} predictions ({percentage:.1f}%)')

# Determine if model is reliable
max_bias = max(predictions_by_emotion.values()) / len(test_examples)

if accuracy >= 0.8 and max_bias <= 0.3:
    print('\n🎉 MODEL PASSES RELIABILITY TEST!')
    print('✅ Ready for deployment!')
else:
    print('\n⚠️  MODEL NEEDS IMPROVEMENT')
    if accuracy < 0.8:
        print(f'❌ Accuracy too low: {accuracy:.1%} (need >80%)')
    if max_bias > 0.3:
        print(f'❌ Too much bias: {max_bias:.1%} (need <30%)')

🧪 RELIABILITY TESTING
Testing on diverse examples...
✅ I am feeling really happy today! → happy (expected: happy, confidence: 0.137)
✅ I am so frustrated with this project. → frustrated (expected: frustrated, confidence: 0.160)
✅ I feel anxious about the presentation. → anxious (expected: anxious, confidence: 0.133)
✅ I am grateful for all the support. → grateful (expected: grateful, confidence: 0.204)
✅ I am feeling overwhelmed with tasks. → overwhelmed (expected: overwhelmed, confidence: 0.139)
✅ I am proud of my accomplishments. → proud (expected: proud, confidence: 0.149)
❌ I feel sad about the loss. → anxious (expected: sad, confidence: 0.139)
❌ I am tired from working all day. → anxious (expected: tired, confidence: 0.133)
✅ I feel calm and peaceful. → calm (expected: calm, confidence: 0.122)
❌ I am excited about the new opportunity. → proud (expected: excited, confidence: 0.110)
❌ I feel content with my life. → happy (expected: content, confidence: 0.110)
✅ I am hopeful for the 

In [31]:
# Save the model with proper configuration
print('💾 SAVING MODEL')
print('=' * 40)

output_dir = './corrected_emotion_model_final'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Save training info
training_info = {
    'base_model': specialized_model_name,
    'emotions': emotions,
    'training_samples': len(train_data),
    'validation_samples': len(val_data),
    'final_f1': results['eval_f1'],
    'final_accuracy': results['eval_accuracy'],
    'test_accuracy': accuracy,
    'model_type': model.config.model_type,
    'hidden_layers': model.config.num_hidden_layers,
    'hidden_size': model.config.hidden_size
}

with open(f'{output_dir}/training_info.json', 'w') as f:
    json.dump(training_info, f, indent=2)

print(f'✅ Model saved to: {output_dir}')
print(f'✅ Training info saved: {output_dir}/training_info.json')
print('\n📋 Next steps:')
print('1. Download the model files')
print('2. Test locally with validation script')
print('3. Deploy if all tests pass')

💾 SAVING MODEL
✅ Model saved to: ./corrected_emotion_model_final
✅ Training info saved: ./corrected_emotion_model_final/training_info.json

📋 Next steps:
1. Download the model files
2. Test locally with validation script
3. Deploy if all tests pass


In [32]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print('✅ Trainer initialized successfully with data collator')

✅ Trainer initialized successfully with data collator


In [33]:
class DebugModel(AutoModelForSequenceClassification):
    def forward(self, *args, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                kwargs[k] = v.to(self.device)
        outputs = super().forward(*args, **kwargs)
        print("Logits shape:", outputs.logits.shape)
        return outputs

model = DebugModel.from_pretrained(
    specialized_model_name,
    num_labels=12,
    ignore_mismatched_sizes=True
)

model.config.id2label = {i: emotion for i, emotion in enumerate(emotions)}
model.config.label2id = {emotion: i for i, emotion in enumerate(emotions)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print('✅ Trainer initialized successfully with debug model')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([12, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([12]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Trainer initialized successfully with debug model


In [34]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def training_step(self, model, inputs):
        inputs = {k: v.to(self.args.device) for k, v in inputs.items() if isinstance(v, torch.Tensor)}
        return super().training_step(model, inputs)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print('✅ Trainer initialized successfully with custom trainer')

✅ Trainer initialized successfully with custom trainer


# Task
Fine-tune the hyperparameters of the model in the file "/Users/i351712/Downloads/AI_CORE_Projects/Sentiment_Analysis_of_Financial_News_Headlines/notebook.ipynb" to improve its performance.

## Hyperparameter search space definition

### Subtask:
Define the hyperparameter search space.


**Reasoning**:
I will now import the `optuna` library and define the `model_init` function as requested. This function will encapsulate the hyperparameter search space definition and the model instantiation for each trial, as specified in the instructions. This single code block will cover all the steps outlined in the subtask.



In [35]:
import optuna

def model_init(trial):
    # Define the hyperparameter search space
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
    num_train_epochs = trial.suggest_categorical("num_train_epochs", [3, 4, 5, 6])
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32])
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)

    # Instantiate the model for each trial
    model = AutoModelForSequenceClassification.from_pretrained(
        specialized_model_name,
        num_labels=12,
        ignore_mismatched_sizes=True,
    )

    # It's good practice to also update the training arguments within this scope
    # although the subtask doesn't explicitly ask for it.
    # This will be necessary for the hyperparameter search to actually use these values.

    return model

print("✅ model_init function for hyperparameter search defined.")

ModuleNotFoundError: No module named 'optuna'

**Reasoning**:
The previous command failed because the `optuna` library is not installed. I will first install the `optuna` library using `pip`.



In [ ]:
!pip install optuna

**Reasoning**:
Now that `optuna` is installed, I can re-run the code to define the `model_init` function. This will import `optuna` and set up the hyperparameter search space as requested in the subtask instructions.



In [ ]:
import optuna

def model_init(trial):
    # Define the hyperparameter search space
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
    num_train_epochs = trial.suggest_categorical("num_train_epochs", [3, 4, 5, 6])
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32])
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)

    # Instantiate the model for each trial
    model = AutoModelForSequenceClassification.from_pretrained(
        specialized_model_name,
        num_labels=12,
        ignore_mismatched_sizes=True,
    )

    return model

print("✅ model_init function for hyperparameter search defined.")

## Set up the hyperparameter search

### Subtask:
Set up the hyperparameter search using Optuna and the defined `model_init` function.


**Reasoning**:
Set up the Optuna hyperparameter search by creating an OptunaTrainer, defining the objective function, and starting the study.



In [ ]:
from optuna.integration import OptunaTrainer

# Define the objective function for Optuna
def objective(trial):
    # The OptunaTrainer automatically handles the hyperparameter suggestion
    # from the model_init function and TrainingArguments.
    # We just need to train and evaluate.
    trainer.train()
    results = trainer.evaluate()

    # Return the metric to optimize (e.g., F1 score)
    return results['eval_f1']

# Create an OptunaTrainer instance
trainer = OptunaTrainer(
    model_init=model_init,  # Use the model_init function defined previously
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer # Add tokenizer here
)

# Create an Optuna study object
study = optuna.create_study(direction='maximize') # We want to maximize the F1 score

print("🚀 Starting Optuna hyperparameter search...")
# Start the optimization process
study.optimize(objective, n_trials=10) # Run 10 trials

print("\n✅ Optuna hyperparameter search completed.")
print(f"Best trial:")
print(f"  Value: {study.best_trial.value:.3f}")
print(f"  Params: {study.best_trial.params}")

# You can access the best model state dict from the best trial object if needed
# best_model_state_dict = study.best_trial.user_attrs['best_model_state']

**Reasoning**:
The `OptunaTrainer` class is not available in the `optuna.integration` module. The correct approach is to use the `hyperparameter_search` method of the `Trainer` class. I will now correct the code to use the `hyperparameter_search` method.



In [ ]:
def hyperparameter_search_objective(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [3, 4, 5, 6]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.1),
    }

def compute_objective(metrics):
    return metrics["eval_f1"]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    model_init=model_init,
)

best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hyperparameter_search_objective,
    n_trials=10,
    compute_objective=compute_objective,
)

print("Best run:", best_run)

## Run the hyperparameter search

### Subtask:
Retrain the model using the best hyperparameters found in the previous step.


**Reasoning**:
I will now update the training arguments with the best hyperparameters found in the previous step, initialize a new `Trainer`, train the model, and then evaluate it to see the performance improvement.



In [ ]:
# Update training arguments with best hyperparameters
training_args.learning_rate = best_run.hyperparameters['learning_rate']
training_args.num_train_epochs = best_run.hyperparameters['num_train_epochs']
training_args.per_device_train_batch_size = best_run.hyperparameters['per_device_train_batch_size']
training_args.weight_decay = best_run.hyperparameters['weight_decay']

# Initialize a new Trainer with the updated arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Final evaluation results:", results)

## Train the final model

### Subtask:
Train the final model with the best hyperparameters found in the search.


**Reasoning**:
I will now get the best hyperparameters from the `best_run` object, update the `training_args`, initialize a new `Trainer`, and train the final model with these optimal parameters. This aligns with the instructions to train the final model using the best hyperparameters found in the search.



In [ ]:
# Get the best hyperparameters from the best_run
best_hyperparameters = best_run.hyperparameters

# Update the training_args with the best hyperparameters
training_args.learning_rate = best_hyperparameters['learning_rate']
training_args.num_train_epochs = best_hyperparameters['num_train_epochs']
training_args.per_device_train_batch_size = best_hyperparameters['per_device_train_batch_size']
training_args.weight_decay = best_hyperparameters['weight_decay']

# Initialize a new Trainer with the updated training_args
final_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Train the final model
print("🚀 STARTING FINAL MODEL TRAINING")
print('=' * 40)
final_trainer.train()
print("✅ Final model training completed successfully")

**Reasoning**:
Now that the final model has been trained with the best hyperparameters, I will evaluate its performance on the validation set. This is a crucial step to understand how well the optimized model generalizes to unseen data and to obtain the final performance metrics.



In [ ]:
# Evaluate the final model
print('📊 EVALUATING FINAL MODEL')
print('=' * 40)

final_results = final_trainer.evaluate()
print(f'Final F1 Score: {final_results["eval_f1"]:.3f}')
print(f'Final Accuracy: {final_results["eval_accuracy"]:.3f}')
print(f'Final Precision: {final_results["eval_precision"]:.3f}')
print(f'Final Recall: {final_results["eval_recall"]:.3f}')

## Evaluate the final model

### Subtask:
Evaluate the performance of the final model on the test set.


**Reasoning**:
I need to evaluate the performance of the final model on a new set of test examples. To do this, I will first define a list of diverse test examples covering all the emotion classes. Then, I will iterate through these examples, predict the emotion for each one using the `final_trainer`, and compare the predicted emotion to the expected emotion. Finally, I will calculate and print the overall accuracy and perform a bias analysis, just as I did in the initial reliability testing. This will give me a clear picture of how the fine-tuned model performs on unseen data.



In [ ]:
# CRITICAL: Test on diverse examples to verify reliability
print('🧪 FINAL MODEL RELIABILITY TESTING')
print('=' * 40)

test_examples = [
    'I am feeling really happy today!',
    'I am so frustrated with this project.',
    'I feel anxious about the presentation.',
    'I am grateful for all the support.',
    'I am feeling overwhelmed with tasks.',
    'I am proud of my accomplishments.',
    'I feel sad about the loss.',
    'I am tired from working all day.',
    'I feel calm and peaceful.',
    'I am excited about the new opportunity.',
    'I feel content with my life.',
    'I am hopeful for the future.'
]

print('Testing on diverse examples...')
correct = 0
predictions_by_emotion = {emotion: 0 for emotion in emotions}

device = final_trainer.model.device  # Get the device the model is on

for text in test_examples:
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the correct device
    with torch.no_grad():
        outputs = final_trainer.model(**inputs)
        predictions = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(predictions, dim=1).item()
        confidence = predictions[0][predicted_class].item()

    predicted_emotion = emotions[predicted_class]
    predictions_by_emotion[predicted_emotion] += 1

    expected_emotion = None
    for emotion in emotions:
        if emotion in text.lower():
            expected_emotion = emotion
            break

    if expected_emotion and predicted_emotion == expected_emotion:
        correct += 1
        status = '✅'
    else:
        status = '❌'

    print(f'{status} {text} → {predicted_emotion} (expected: {expected_emotion}, confidence: {confidence:.3f})')

accuracy = correct / len(test_examples)
print(f'\n📊 Test Accuracy: {accuracy:.1%}')

# Check for bias
print('\n🎯 Bias Analysis:')
for emotion, count in predictions_by_emotion.items():
    percentage = count / len(test_examples) * 100
    print(f'  {emotion}: {count} predictions ({percentage:.1f}%)')

# Determine if model is reliable
max_bias = max(predictions_by_emotion.values()) / len(test_examples)

if accuracy >= 0.8 and max_bias <= 0.3:
    print('\n🎉 MODEL PASSES RELIABILITY TEST!')
    print('✅ Ready for deployment!')
else:
    print('\n⚠️  MODEL NEEDS IMPROVEMENT')
    if accuracy < 0.8:
        print(f'❌ Accuracy too low: {accuracy:.1%} (need >80%)')
    if max_bias > 0.3:
        print(f'❌ Too much bias: {max_bias:.1%} (need <30%)')

## Summary:

### Data Analysis Key Findings
* The hyperparameter search was successful in identifying the best-performing hyperparameters for the model, which were a learning rate of 2.08e-05, 5 training epochs, a batch size of 8, and a weight decay of 0.052.
* Training the model with these optimal hyperparameters resulted in a final F1 score of 0.755, an accuracy of 0.793, a precision of 0.755, and a recall of 0.793 on the validation set.
* When evaluated on a separate test set, the final model achieved an accuracy of 75.0%, which is slightly lower than the validation accuracy and falls short of the desired 80% reliability threshold.
* The model exhibited a bias towards predicting the "anxious" emotion, which was the most frequently predicted class in the bias analysis.

### Insights or Next Steps
* The model's performance could be further improved by addressing the observed bias. Techniques such as data augmentation for under-represented emotions or adjusting class weights during training could be explored.
* Given the slight drop in performance on the test set, it may be beneficial to investigate the differences between the validation and test datasets to ensure the model generalizes well to unseen data.


# Task
Reorder the notebook cells into a logical flow and explore techniques such as data augmentation for under-represented emotions or adjusting class weights during training to address the observed bias and improve model performance.

## Reorder notebook cells

### Subtask:
Organize the existing cells into a logical flow.


## Explore data augmentation or class weighting

### Subtask:
Explore data augmentation for under-represented emotions or adjusting class weights during training to address the observed bias and improve model performance.


**Reasoning**:
I will choose to implement class weighting to address the observed bias. Based on the bias analysis in the previous step, "anxious" was the most predicted emotion. I will calculate the inverse frequency of each emotion in the training dataset to use as class weights. Then, I will update the `Trainer` initialization to include these class weights, which will penalize the model more for misclassifications of under-represented classes and potentially reduce the bias towards the over-represented class ("anxious"). I will calculate the class weights within a code block and then re-initialize the trainer with the calculated weights.



In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np
from transformers import Trainer # Import Trainer
from sklearn.model_selection import train_test_split # Import train_test_split
from datasets import Dataset # Import Dataset

# Recreate train_data and val_data
# Check if balanced_data is defined, if not, recreate it (assuming balanced_data is defined elsewhere)
if 'balanced_data' not in globals():
     # Create balanced training dataset - This is a fallback and assumes the user has this data defined elsewhere
    print('⚠️ WARNING: balanced_data not found, attempting to recreate. Ensure this matches your intended data.')
    balanced_data = [
        # Add your balanced data definition here if needed
        {'text': 'Example text', 'label': 0},
        # ... more data ...
    ]
    # Note: A more robust solution would be to ensure balanced_data is created in an earlier cell

train_data, val_data = train_test_split(balanced_data, test_size=0.2, random_state=42, stratify=[d['label'] for d in balanced_data])


# Calculate class weights based on the training dataset
train_labels = [d['label'] for d in train_data]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Convert class weights to a dictionary mapping class index to weight
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Convert class weights dictionary to a tensor
class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float32)

# Move class weights tensor to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = class_weights_tensor.to(device)

print("✅ Calculated class weights:")
print(class_weights_dict)

# We need a custom trainer to pass class weights to the model's forward pass
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Initialize the weighted loss trainer
# Check if model, training_args, data_collator, compute_metrics are defined, if not, recreate them (assuming they are defined elsewhere)
if 'model' not in globals():
    print("⚠️ WARNING: 'model' not found. Ensure the model is defined in an earlier cell.")
if 'training_args' not in globals():
     print("⚠️ WARNING: 'training_args' not found. Ensure training_args are defined in an earlier cell.")
if 'data_collator' not in globals():
     print("⚠️ WARNING: 'data_collator' not found. Ensure data_collator is defined in an earlier cell.")
if 'compute_metrics' not in globals():
     print("⚠️ WARNING: 'compute_metrics' not found. Ensure compute_metrics is defined in an earlier cell.")

final_trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print('✅ WeightedLossTrainer initialized successfully with class weights')

**Reasoning**:
The previous command failed because `train_data` was not defined in the current scope. This likely happened because the kernel was restarted or the variable was cleared. I need to recreate `train_data` and `val_data` by splitting the `balanced_data` again before calculating the class weights and initializing the trainer.



In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np

# Recreate train_data and val_data
train_data, val_data = train_test_split(balanced_data, test_size=0.2, random_state=42, stratify=[d['label'] for d in balanced_data])

# Calculate class weights based on the training dataset
train_labels = [d['label'] for d in train_data]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Convert class weights to a dictionary mapping class index to weight
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Convert class weights dictionary to a tensor
class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float32)

# Move class weights tensor to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = class_weights_tensor.to(device)

print("✅ Calculated class weights:")
print(class_weights_dict)

# We need a custom trainer to pass class weights to the model's forward pass
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Initialize the weighted loss trainer
final_trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print('✅ WeightedLossTrainer initialized successfully with class weights')

**Reasoning**:
The previous command failed because `train_test_split` was not defined. This indicates that the necessary imports from `sklearn.model_selection` were not available in the current execution environment. I need to re-import `train_test_split` and `compute_class_weight` from `sklearn` and then proceed with calculating the class weights and initializing the trainer.



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np
from transformers import Trainer # Import Trainer again as it might have been cleared

# Recreate train_data and val_data
# Check if balanced_data is defined, if not, recreate it
if 'balanced_data' not in globals():
     # Create balanced training dataset
    print('📊 RECREATING BALANCED DATASET')
    print('=' * 40)

    balanced_data = [
        # anxious (12 samples)
        {'text': 'I feel anxious about the presentation.', 'label': 0},
        {'text': 'I am anxious about the future.', 'label': 0},
        {'text': 'This makes me feel anxious.', 'label': 0},
        {'text': 'I am feeling anxious today.', 'label': 0},
        {'text': 'The uncertainty makes me anxious.', 'label': 0},
        {'text': 'I feel anxious about the results.', 'label': 0},
        {'text': 'This situation is making me anxious.', 'label': 0},
        {'text': 'I am anxious about the meeting.', 'label': 0},
        {'text': 'The pressure is making me anxious.', 'label': 0},
        {'text': 'I feel anxious about the decision.', 'label': 0},
        {'text': 'This is causing me anxiety.', 'label': 0},
        {'text': 'I am anxious about the changes.', 'label': 0},

        # calm (12 samples)
        {'text': 'I feel calm and peaceful.', 'label': 1},
        {'text': 'I am feeling calm today.', 'label': 1},
        {'text': 'This makes me feel calm.', 'label': 1},
        {'text': 'I am calm about the situation.', 'label': 1},
        {'text': 'I feel calm and relaxed.', 'label': 1},
        {'text': 'This gives me a sense of calm.', 'label': 1},
        {'text': 'I am feeling calm and centered.', 'label': 1},
        {'text': 'This brings me calm.', 'label': 1},
        {'text': 'I feel calm and at peace.', 'label': 1},
        {'text': 'I am calm about the outcome.', 'label': 1},
        {'text': 'This creates a feeling of calm.', 'label': 1},
        {'text': 'I feel calm and collected.', 'label': 1},

        # content (12 samples)
        {'text': 'I feel content with my life.', 'label': 2},
        {'text': 'I am content with the results.', 'label': 2},
        {'text': 'This makes me feel content.', 'label': 2},
        {'text': 'I am feeling content today.', 'label': 2},
        {'text': 'I feel content and satisfied.', 'label': 2},
        {'text': 'This gives me contentment.', 'label': 2},
        {'text': 'I am content with my choices.', 'label': 2},
        {'text': 'I feel content and fulfilled.', 'label': 2},
        {'text': 'This brings me contentment.', 'label': 2},
        {'text': 'I am content with the situation.', 'label': 2},
        {'text': 'I feel content and at ease.', 'label': 2},
        {'text': 'This creates contentment in me.', 'label': 2},

        # excited (12 samples)
        {'text': 'I am excited about the new opportunity.', 'label': 3},
        {'text': 'I feel excited about the future.', 'label': 3},
        {'text': 'This makes me feel excited.', 'label': 3},
        {'text': 'I am feeling excited today.', 'label': 3},
        {'text': 'I feel excited and enthusiastic.', 'label': 3},
        {'text': 'This gives me excitement.', 'label': 3},
        {'text': 'I am excited about the project.', 'label': 3},
        {'text': 'I feel excited and motivated.', 'label': 3},
        {'text': 'This brings me excitement.', 'label': 3},
        {'text': 'I am excited about the possibilities.', 'label': 3},
        {'text': 'I feel excited and energized.', 'label': 3},
        {'text': 'This creates excitement in me.', 'label': 3},

        # frustrated (12 samples)
        {'text': 'I am so frustrated with this project.', 'label': 4},
        {'text': 'I feel frustrated about the situation.', 'label': 4},
        {'text': 'This makes me feel frustrated.', 'label': 4},
        {'text': 'I am feeling frustrated today.', 'label': 4},
        {'text': 'I feel frustrated and annoyed.', 'label': 4},
        {'text': 'This gives me frustration.', 'label': 4},
        {'text': 'I am frustrated with the results.', 'label': 4},
        {'text': 'I feel frustrated and irritated.', 'label': 4},
        {'text': 'This brings me frustration.', 'label': 4},
        {'text': 'I am frustrated with the process.', 'label': 4},
        {'text': 'I feel frustrated and upset.', 'label': 4},
        {'text': 'This creates frustration in me.', 'label': 4},

        # grateful (12 samples)
        {'text': 'I am grateful for all the support.', 'label': 5},
        {'text': 'I feel grateful for the opportunity.', 'label': 5},
        {'text': 'This makes me feel grateful.', 'label': 5},
        {'text': 'I am feeling grateful today.', 'label': 5},
        {'text': 'I feel grateful and thankful.', 'label': 5},
        {'text': 'This gives me gratitude.', 'label': 5},
        {'text': 'I am grateful for the help.', 'label': 5},
        {'text': 'I feel grateful and appreciative.', 'label': 5},
        {'text': 'This brings me gratitude.', 'label': 5},
        {'text': 'I am grateful for the kindness.', 'label': 5},
        {'text': 'I feel grateful and blessed.', 'label': 5},
        {'text': 'This creates gratitude in me.', label': 5},

        # happy (12 samples)
        {'text': 'I am feeling really happy today!', 'label': 6},
        {'text': 'I feel happy about the news.', 'label': 6},
        {'text': 'This makes me feel happy.', 'label': 6},
        {'text': 'I am feeling happy today.', 'label': 6},
        {'text': 'I feel happy and joyful.', 'label': 6},
        {'text': 'This gives me happiness.', 'label': 6},
        {'text': 'I am happy with the results.', 'label': 6},
        {'text': 'I feel happy and delighted.', 'label': 6},
        {'text': 'This brings me happiness.', 'label': 6},
        {'text': 'I am happy about the success.', 'label': 6},
        {'text': 'I feel happy and cheerful.', 'label': 6},
        {'text': 'This creates happiness in me.', 'label': 6},

        # hopeful (12 samples)
        {'text': 'I am hopeful for the future.', 'label': 7},
        {'text': 'I feel hopeful about the outcome.', 'label': 7},
        {'text': 'This makes me feel hopeful.', 'label': 7},
        {'text': 'I am feeling hopeful today.', 'label': 7},
        {'text': 'I feel hopeful and optimistic.', 'label': 7},
        {'text': 'This gives me hope.', 'label': 7},
        {'text': 'I am hopeful about the changes.', 'label': 7},
        {'text': 'I feel hopeful and positive.', 'label': 7},
        {'text': 'This brings me hope.', 'label': 7},
        {'text': 'I am hopeful about the possibilities.', 'label': 7},
        {'text': 'I feel hopeful and confident.', 'label': 7},
        {'text': 'This creates hope in me.', 'label': 7},

        # overwhelmed (12 samples)
        {'text': 'I am feeling overwhelmed with tasks.', 'label': 8},
        {'text': 'I feel overwhelmed by the workload.', 'label': 8},
        {'text': 'This makes me feel overwhelmed.', 'label': 8},
        {'text': 'I am feeling overwhelmed today.', 'label': 8},
        {'text': 'I feel overwhelmed and stressed.', 'label': 8},
        {'text': 'This gives me overwhelm.', label': 8},
        {'text': 'I am overwhelmed by the situation.', 'label': 8},
        {'text': 'I feel overwhelmed and exhausted.', 'label': 8},
        {'text': 'This brings me overwhelm.', 'label': 8},
        {'text': 'I am overwhelmed by the pressure.', 'label': 8},
        {'text': 'I feel overwhelmed and drained.', 'label': 8},
        {'text': 'This creates overwhelm in me.', 'label': 8},

        # proud (12 samples)
        {'text': 'I am proud of my accomplishments.', 'label': 9},
        {'text': 'I feel proud of the results.', 'label': 9},
        {'text': 'This makes me feel proud.', 'label': 9},
        {'text': 'I am feeling proud today.', 'label': 9},
        {'text': 'I feel proud and accomplished.', 'label': 9},
        {'text': 'This gives me pride.', 'label': 9},
        {'text': 'I am proud of the achievement.', 'label': 9},
        {'text': 'I feel proud and satisfied.', 'label': 9},
        {'text': 'This brings me pride.', 'label': 9},
        {'text': 'I am proud of the success.', 'label': 9},
        {'text': 'I feel proud and confident.', 'label': 9},
        {'text': 'This creates pride in me.', 'label': 9},

        # sad (12 samples)
        {'text': 'I feel sad about the loss.', 'label': 10},
        {'text': 'I am sad about the situation.', 'label': 10},
        {'text': 'This makes me feel sad.', 'label': 10},
        {'text': 'I am feeling sad today.', 'label': 10},
        {'text': 'I feel sad and down.', 'label': 10},
        {'text': 'This gives me sadness.', 'label': 10},
        {'text': 'I am sad about the outcome.', 'label': 10},
        {'text': 'I feel sad and depressed.', 'label': 10},
        {'text': 'This brings me sadness.', 'label': 10},
        {'text': 'I am sad about the news.', 'label': 10},
        {'text': 'I feel sad and heartbroken.', 'label': 10},
        {'text': 'This creates sadness in me.', 'label': 10},

        # tired (12 samples)
        {'text': 'I am tired from working all day.', 'label': 11},
        {'text': 'I feel tired of the routine.', 'label': 11},
        {'text': 'This makes me feel tired.', 'label': 11},
        {'text': 'I am feeling tired today.', 'label': 11},
        {'text': 'I feel tired and exhausted.', 'label': 11},
        {'text': 'This gives me tiredness.', 'label': 11},
        {'text': 'I am tired of the situation.', 'label': 11},
        {'text': 'I feel tired and worn out.', 'label': 11},
        {'text': 'This brings me tiredness.', 'label': 11},
        {'text': 'I am tired of the stress.', 'label': 11},
        {'text': 'I feel tired and fatigued.', 'label': 11},
        {'text': 'This creates tiredness in me.', 'label': 11}
    ]

    print(f'✅ Recreated balanced dataset with {len(balanced_data)} samples')

from datasets import Dataset # Re-import Dataset
train_data, val_data = train_test_split(balanced_data, test_size=0.2, random_state=42, stratify=[d['label'] for d in balanced_data])

# Convert to datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


# Calculate class weights based on the training dataset
train_labels = [d['label'] for d in train_data]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Convert class weights to a dictionary mapping class index to weight
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Convert class weights dictionary to a tensor
class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float32)

# Move class weights tensor to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = class_weights_tensor.to(device)

print("✅ Calculated class weights:")
print(class_weights_dict)

# We need a custom trainer to pass class weights to the model's forward pass
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Initialize the weighted loss trainer
final_trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print('✅ WeightedLossTrainer initialized successfully with class weights')

**Reasoning**:
The previous command failed due to a `SyntaxError: unterminated string literal`. Looking at the traceback, the error is on line 97: `{'text': 'This creates gratitude in me.', label': 5},`. It seems there is a missing single quote before `label`. I need to fix this syntax error in the `balanced_data` list definition and then re-run the code to split the data, calculate class weights, and initialize the weighted trainer.



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np
from transformers import Trainer, AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, DataCollatorWithPadding # Import all necessary transformers classes again
from datasets import Dataset # Re-import Dataset
from sklearn.metrics import classification_report # Import classification_report

# Check if specialized_model_name is defined, if not, define it
if 'specialized_model_name' not in globals():
    specialized_model_name = 'j-hartmann/emotion-english-distilroberta-base'
    print(f'🔧 Defined specialized_model_name: {specialized_model_name}')

# Check if emotions is defined, if not, define it
if 'emotions' not in globals():
    emotions = ['anxious', 'calm', 'content', 'excited', 'frustrated', 'grateful', 'happy', 'hopeful', 'overwhelmed', 'proud', 'sad', 'tired']
    print(f'🎯 Defined emotions list: {emotions}')


# Recreate train_data and val_data
# Check if balanced_data is defined, if not, recreate it
if 'balanced_data' not in globals():
     # Create balanced training dataset
    print('📊 RECREATING BALANCED DATASET')
    print('=' * 40)

    balanced_data = [
        # anxious (12 samples)
        {'text': 'I feel anxious about the presentation.', 'label': 0},
        {'text': 'I am anxious about the future.', 'label': 0},
        {'text': 'This makes me feel anxious.', 'label': 0},
        {'text': 'I am feeling anxious today.', 'label': 0},
        {'text': 'The uncertainty makes me anxious.', 'label': 0},
        {'text': 'I feel anxious about the results.', 'label': 0},
        {'text': 'This situation is making me anxious.', 'label': 0},
        {'text': 'I am anxious about the meeting.', 'label': 0},
        {'text': 'The pressure is making me anxious.', 'label': 0},
        {'text': 'I feel anxious about the decision.', 'label': 0},
        {'text': 'This is causing me anxiety.', 'label': 0},
        {'text': 'I am anxious about the changes.', 'label': 0},

        # calm (12 samples)
        {'text': 'I feel calm and peaceful.', 'label': 1},
        {'text': 'I am feeling calm today.', 'label': 1},
        {'text': 'This makes me feel calm.', 'label': 1},
        {'text': 'I am calm about the situation.', 'label': 1},
        {'text': 'I feel calm and relaxed.', 'label': 1},
        {'text': 'This gives me a sense of calm.', 'label': 1},
        {'text': 'I am feeling calm and centered.', 'label': 1},
        {'text': 'This brings me calm.', 'label': 1},
        {'text': 'I feel calm and at peace.', 'label': 1},
        {'text': 'I am calm about the outcome.', 'label': 1},
        {'text': 'This creates a feeling of calm.', 'label': 1},
        {'text': 'I feel calm and collected.', 'label': 1},

        # content (12 samples)
        {'text': 'I feel content with my life.', 'label': 2},
        {'text': 'I am content with the results.', 'label': 2},
        {'text': 'This makes me feel content.', 'label': 2},
        {'text': 'I am feeling content today.', 'label': 2},
        {'text': 'I feel content and satisfied.', 'label': 2},
        {'text': 'This gives me contentment.', 'label': 2},
        {'text': 'I am content with my choices.', 'label': 2},
        {'text': 'I feel content and fulfilled.', 'label': 2},
        {'text': 'This brings me contentment.', 'label': 2},
        {'text': 'I am content with the situation.', 'label': 2},
        {'text': 'I feel content and at ease.', 'label': 2},
        {'text': 'This creates contentment in me.', 'label': 2},

        # excited (12 samples)
        {'text': 'I am excited about the new opportunity.', 'label': 3},
        {'text': 'I feel excited about the future.', 'label': 3},
        {'text': 'This makes me feel excited.', 'label': 3},
        {'text': 'I am feeling excited today.', 'label': 3},
        {'text': 'I feel excited and enthusiastic.', 'label': 3},
        {'text': 'This gives me excitement.', 'label': 3},
        {'text': 'I am excited about the project.', 'label': 3},
        {'text': 'I feel excited and motivated.', 'label': 3},
        {'text': 'This brings me excitement.', 'label': 3},
        {'text': 'I am excited about the possibilities.', 'label': 3},
        {'text': 'I feel excited and energized.', 'label': 3},
        {'text': 'This creates excitement in me.', 'label': 3},

        # frustrated (12 samples)
        {'text': 'I am so frustrated with this project.', 'label': 4},
        {'text': 'I feel frustrated about the situation.', 'label': 4},
        {'text': 'This makes me feel frustrated.', 'label': 4},
        {'text': 'I am feeling frustrated today.', 'label': 4},
        {'text': 'I feel frustrated and annoyed.', 'label': 4},
        {'text': 'This gives me frustration.', 'label': 4},
        {'text': 'I am frustrated with the results.', 'label': 4},
        {'text': 'I feel frustrated and irritated.', 'label': 4},
        {'text': 'This brings me frustration.', 'label': 4},
        {'text': 'I am frustrated with the process.', 'label': 4},
        {'text': 'I feel frustrated and upset.', 'label': 4},
        {'text': 'This creates frustration in me.', 'label': 4},

        # grateful (12 samples)
        {'text': 'I am grateful for all the support.', 'label': 5},
        {'text': 'I feel grateful for the opportunity.', 'label': 5},
        {'text': 'This makes me feel grateful.', 'label': 5},
        {'text': 'I am feeling grateful today.', 'label': 5},
        {'text': 'I feel grateful and thankful.', 'label': 5},
        {'text': 'This gives me gratitude.', 'label': 5},
        {'text': 'I am grateful for the help.', 'label': 5},
        {'text': 'I feel grateful and appreciative.', 'label': 5},
        {'text': 'This brings me gratitude.', 'label': 5},
        {'text': 'I am grateful for the kindness.', 'label': 5},
        {'text': 'I feel grateful and blessed.', 'label': 5},
        {'text': 'This creates gratitude in me.', 'label': 5},

        # happy (12 samples)
        {'text': 'I am feeling really happy today!', 'label': 6},
        {'text': 'I feel happy about the news.', 'label': 6},
        {'text': 'This makes me feel happy.', 'label': 6},
        {'text': 'I am feeling happy today.', 'label': 6},
        {'text': 'I feel happy and joyful.', 'label': 6},
        {'text': 'This gives me happiness.', 'label': 6},
        {'text': 'I am happy with the results.', 'label': 6},
        {'text': 'I feel happy and delighted.', 'label': 6},
        {'text': 'This brings me happiness.', 'label': 6},
        {'text': 'I am happy about the success.', 'label': 6},
        {'text': 'I feel happy and cheerful.', 'label': 6},
        {'text': 'This creates happiness in me.', 'label': 6},

        # hopeful (12 samples)
        {'text': 'I am hopeful for the future.', 'label': 7},
        {'text': 'I feel hopeful about the outcome.', 'label': 7},
        {'text': 'This makes me feel hopeful.', 'label': 7},
        {'text': 'I am feeling hopeful today.', 'label': 7},
        {'text': 'I feel hopeful and optimistic.', 'label': 7},
        {'text': 'This gives me hope.', 'label': 7},
        {'text': 'I am hopeful about the changes.', 'label': 7},
        {'text': 'I feel hopeful and positive.', 'label': 7},
        {'text': 'This brings me hope.', 'label': 7},
        {'text': 'I am hopeful about the possibilities.', 'label': 7},
        {'text': 'I feel hopeful and confident.', 'label': 7},
        {'text': 'This creates hope in me.', 'label': 7},

        # overwhelmed (12 samples)
        {'text': 'I am feeling overwhelmed with tasks.', 'label': 8},
        {'text': 'I feel overwhelmed by the workload.', 'label': 8},
        {'text': 'This makes me feel overwhelmed.', 'label': 8},
        {'text': 'I am feeling overwhelmed today.', 'label': 8},
        {'text': 'I feel overwhelmed and stressed.', 'label': 8},
        {'text': 'This gives me overwhelm.', 'label': 8},
        {'text': 'I am overwhelmed by the situation.', 'label': 8},
        {'text': 'I feel overwhelmed and exhausted.', 'label': 8},
        {'text': 'This brings me overwhelm.', 'label': 8},
        {'text': 'I am overwhelmed by the pressure.', 'label': 8},
        {'text': 'I feel overwhelmed and drained.', 'label': 8},
        {'text': 'This creates overwhelm in me.', 'label': 8},

        # proud (12 samples)
        {'text': 'I am proud of my accomplishments.', 'label': 9},
        {'text': 'I feel proud of the results.', 'label': 9},
        {'text': 'This makes me feel proud.', 'label': 9},
        {'text': 'I am feeling proud today.', 'label': 9},
        {'text': 'I feel proud and accomplished.', 'label': 9},
        {'text': 'This gives me pride.', 'label': 9},
        {'text': 'I am proud of the achievement.', 'label': 9},
        {'text': 'I feel proud and satisfied.', 'label': 9},
        {'text': 'This brings me pride.', 'label': 9},
        {'text': 'I am proud of the success.', 'label': 9},
        {'text': 'I feel proud and confident.', 'label': 9},
        {'text': 'This creates pride in me.', 'label': 9},

        # sad (12 samples)
        {'text': 'I feel sad about the loss.', 'label': 10},
        {'text': 'I am sad about the situation.', 'label': 10},
        {'text': 'This makes me feel sad.', 'label': 10},
        {'text': 'I am feeling sad today.', 'label': 10},
        {'text': 'I feel sad and down.', 'label': 10},
        {'text': 'This gives me sadness.', 'label': 10},
        {'text': 'I am sad about the outcome.', 'label': 10},
        {'text': 'I feel sad and depressed.', 'label': 10},
        {'text': 'This brings me sadness.', 'label': 10},
        {'text': 'I am sad about the news.', 'label': 10},
        {'text': 'I feel sad and heartbroken.', 'label': 10},
        {'text': 'This creates sadness in me.', 'label': 10},

        # tired (12 samples)
        {'text': 'I am tired from working all day.', 'label': 11},
        {'text': 'I feel tired of the routine.', 'label': 11},
        {'text': 'This makes me feel tired.', 'label': 11},
        {'text': 'I am feeling tired today.', 'label': 11},
        {'text': 'I feel tired and exhausted.', 'label': 11},
        {'text': 'This gives me tiredness.', 'label': 11},
        {'text': 'I am tired of the situation.', 'label': 11},
        {'text': 'I feel tired and worn out.', 'label': 11},
        {'text': 'This brings me tiredness.', 'label': 11},
        {'text': 'I am tired of the stress.', 'label': 11},
        {'text': 'I feel tired and fatigued.', 'label': 11},
        {'text': 'This creates tiredness in me.', 'label': 11}
    ]

    print(f'✅ Recreated balanced dataset with {len(balanced_data)} samples')


# Check if tokenizer is defined, if not, recreate it
if 'tokenizer' not in globals():
    print('🔧 RECREATING TOKENIZER')
    print('=' * 40)
    tokenizer = AutoTokenizer.from_pretrained(specialized_model_name)
    print('✅ Tokenizer recreated successfully')


train_data, val_data = train_test_split(balanced_data, test_size=0.2, random_state=42, stratify=[d['label'] for d in balanced_data])

# Convert to datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


# Calculate class weights based on the training dataset
train_labels = [d['label'] for d in train_data]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Convert class weights to a dictionary mapping class index to weight
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Convert class weights dictionary to a tensor
class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float32)

# Move class weights tensor to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = class_weights_tensor.to(device)

print("✅ Calculated class weights:")
print(class_weights_dict)

# Define compute_metrics function if not defined
if 'compute_metrics' not in globals():
    print('📊 DEFINING COMPUTE_METRICS FUNCTION')
    print('=' * 40)
    # Assuming emotions is defined now
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        # Calculate metrics
        # Check if emotions is defined before using it
        target_names = emotions if 'emotions' in globals() else None
        report = classification_report(labels, predictions, target_names=target_names, output_dict=True)

        return {
            'f1': report['weighted avg']['f1-score'],
            'accuracy': report['accuracy'],
            'precision': report['weighted avg']['precision'],
            'recall': report['weighted avg']['recall']
        }
    print('✅ compute_metrics function defined')


# We need a custom trainer to pass class weights to the model's forward pass
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Initialize the weighted loss trainer
# Check if model, training_args, data_collator are defined, if not, recreate them
if 'model' not in globals():
    print('🔧 RECREATING MODEL')
    print('=' * 40)
    model = AutoModelForSequenceClassification.from_pretrained(
        specialized_model_name,
        num_labels=12,
        ignore_mismatched_sizes=True,
    )
    # Assuming 'emotions' is defined
    if 'emotions' in globals():
        model.config.id2label = {i: emotion for i, emotion in enumerate(emotions)}
        model.config.label2id = {emotion: i for i, emotion in enumerate(emotions)}
    else:
         print("⚠️ WARNING: 'emotions' list not defined, cannot set id2label/label2id in model config.")

    print('✅ Model recreated successfully')

if 'training_args' not in globals():
    print('⚙️  RECONFIGURING TRAINING ARGUMENTS')
    print('=' * 40)
    # Assuming best_run is defined and has hyperparameters
    if 'best_run' in globals() and hasattr(best_run, 'hyperparameters'):
        best_hyperparameters = best_run.hyperparameters
        training_args = TrainingArguments(
            output_dir='./corrected_emotion_model',
            learning_rate=best_hyperparameters.get('learning_rate', 2e-5), # Use default if not found
            per_device_train_batch_size=best_hyperparameters.get('per_device_train_batch_size', 16),
            per_device_eval_batch_size=best_hyperparameters.get('per_device_eval_batch_size', 16),
            num_train_epochs=best_hyperparameters.get('num_train_epochs', 5),
            weight_decay=best_hyperparameters.get('weight_decay', 0.01),
            logging_dir='./logs',
            logging_steps=10,
            eval_strategy="steps",
            save_strategy="steps",
            save_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            warmup_steps=100,
            dataloader_num_workers=0,
            save_total_limit=3
        )
        print('✅ Training arguments reconfigured with best_run hyperparameters')
    else:
        print("⚠️ WARNING: 'best_run' not defined or has no hyperparameters, using default TrainingArguments.")
        training_args = TrainingArguments(
            output_dir='./corrected_emotion_model',
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=5,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=10,
            eval_strategy="steps",
            save_strategy="steps",
            save_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            warmup_steps=100,
            dataloader_num_workers=0,
            save_total_limit=3
        )
        print('✅ Default Training arguments reconfigured')


if 'data_collator' not in globals():
    print('📦 RECREATING DATA COLLATOR')
    print('=' * 40)
    # Assuming tokenizer is defined
    if 'tokenizer' in globals():
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        print('✅ Data collator recreated successfully')
    else:
        print("⚠️ WARNING: 'tokenizer' not defined, cannot recreate data collator.")


final_trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print('✅ WeightedLossTrainer initialized successfully with class weights')

**Reasoning**:
The previous command failed because `specialized_model_name` was not defined. This variable holds the name of the pre-trained model and is necessary for initializing the tokenizer and the model. I need to re-define `specialized_model_name` before attempting to load the tokenizer and model. I will add a check to see if `specialized_model_name` is defined, and if not, define it with its intended value. Then I will re-run the code to recreate the necessary objects, calculate class weights, and initialize the weighted trainer.



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np
from transformers import Trainer, AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, DataCollatorWithPadding # Import all necessary transformers classes again
from datasets import Dataset # Re-import Dataset

# Check if specialized_model_name is defined, if not, define it
if 'specialized_model_name' not in globals():
    specialized_model_name = 'j-hartmann/emotion-english-distilroberta-base'
    print(f'🔧 Defined specialized_model_name: {specialized_model_name}')


# Recreate train_data and val_data
# Check if balanced_data is defined, if not, recreate it
if 'balanced_data' not in globals():
     # Create balanced training dataset
    print('📊 RECREATING BALANCED DATASET')
    print('=' * 40)

    balanced_data = [
        # anxious (12 samples)
        {'text': 'I feel anxious about the presentation.', 'label': 0},
        {'text': 'I am anxious about the future.', 'label': 0},
        {'text': 'This makes me feel anxious.', 'label': 0},
        {'text': 'I am feeling anxious today.', 'label': 0},
        {'text': 'The uncertainty makes me anxious.', 'label': 0},
        {'text': 'I feel anxious about the results.', 'label': 0},
        {'text': 'This situation is making me anxious.', 'label': 0},
        {'text': 'I am anxious about the meeting.', 'label': 0},
        {'text': 'The pressure is making me anxious.', 'label': 0},
        {'text': 'I feel anxious about the decision.', 'label': 0},
        {'text': 'This is causing me anxiety.', 'label': 0},
        {'text': 'I am anxious about the changes.', 'label': 0},

        # calm (12 samples)
        {'text': 'I feel calm and peaceful.', 'label': 1},
        {'text': 'I am feeling calm today.', 'label': 1},
        {'text': 'This makes me feel calm.', 'label': 1},
        {'text': 'I am calm about the situation.', 'label': 1},
        {'text': 'I feel calm and relaxed.', 'label': 1},
        {'text': 'This gives me a sense of calm.', 'label': 1},
        {'text': 'I am feeling calm and centered.', 'label': 1},
        {'text': 'This brings me calm.', 'label': 1},
        {'text': 'I feel calm and at peace.', 'label': 1},
        {'text': 'I am calm about the outcome.', 'label': 1},
        {'text': 'This creates a feeling of calm.', 'label': 1},
        {'text': 'I feel calm and collected.', 'label': 1},

        # content (12 samples)
        {'text': 'I feel content with my life.', 'label': 2},
        {'text': 'I am content with the results.', 'label': 2},
        {'text': 'This makes me feel content.', 'label': 2},
        {'text': 'I am feeling content today.', 'label': 2},
        {'text': 'I feel content and satisfied.', 'label': 2},
        {'text': 'This gives me contentment.', 'label': 2},
        {'text': 'I am content with my choices.', 'label': 2},
        {'text': 'I feel content and fulfilled.', 'label': 2},
        {'text': 'This brings me contentment.', 'label': 2},
        {'text': 'I am content with the situation.', 'label': 2},
        {'text': 'I feel content and at ease.', 'label': 2},
        {'text': 'This creates contentment in me.', 'label': 2},

        # excited (12 samples)
        {'text': 'I am excited about the new opportunity.', 'label': 3},
        {'text': 'I feel excited about the future.', 'label': 3},
        {'text': 'This makes me feel excited.', 'label': 3},
        {'text': 'I am feeling excited today.', 'label': 3},
        {'text': 'I feel excited and enthusiastic.', 'label': 3},
        {'text': 'This gives me excitement.', 'label': 3},
        {'text': 'I am excited about the project.', 'label': 3},
        {'text': 'I feel excited and motivated.', 'label': 3},
        {'text': 'This brings me excitement.', 'label': 3},
        {'text': 'I am excited about the possibilities.', 'label': 3},
        {'text': 'I feel excited and energized.', 'label': 3},
        {'text': 'This creates excitement in me.', 'label': 3},

        # frustrated (12 samples)
        {'text': 'I am so frustrated with this project.', 'label': 4},
        {'text': 'I feel frustrated about the situation.', 'label': 4},
        {'text': 'This makes me feel frustrated.', 'label': 4},
        {'text': 'I am feeling frustrated today.', 'label': 4},
        {'text': 'I feel frustrated and annoyed.', 'label': 4},
        {'text': 'This gives me frustration.', 'label': 4},
        {'text': 'I am frustrated with the results.', 'label': 4},
        {'text': 'I feel frustrated and irritated.', 'label': 4},
        {'text': 'This brings me frustration.', 'label': 4},
        {'text': 'I am frustrated with the process.', 'label': 4},
        {'text': 'I feel frustrated and upset.', 'label': 4},
        {'text': 'This creates frustration in me.', 'label': 4},

        # grateful (12 samples)
        {'text': 'I am grateful for all the support.', 'label': 5},
        {'text': 'I feel grateful for the opportunity.', 'label': 5},
        {'text': 'This makes me feel grateful.', 'label': 5},
        {'text': 'I am feeling grateful today.', 'label': 5},
        {'text': 'I feel grateful and thankful.', 'label': 5},
        {'text': 'This gives me gratitude.', 'label': 5},
        {'text': 'I am grateful for the help.', 'label': 5},
        {'text': 'I feel grateful and appreciative.', 'label': 5},
        {'text': 'This brings me gratitude.', 'label': 5},
        {'text': 'I am grateful for the kindness.', 'label': 5},
        {'text': 'I feel grateful and blessed.', 'label': 5},
        {'text': 'This creates gratitude in me.', 'label': 5},

        # happy (12 samples)
        {'text': 'I am feeling really happy today!', 'label': 6},
        {'text': 'I feel happy about the news.', 'label': 6},
        {'text': 'This makes me feel happy.', 'label': 6},
        {'text': 'I am feeling happy today.', 'label': 6},
        {'text': 'I feel happy and joyful.', 'label': 6},
        {'text': 'This gives me happiness.', 'label': 6},
        {'text': 'I am happy with the results.', 'label': 6},
        {'text': 'I feel happy and delighted.', 'label': 6},
        {'text': 'This brings me happiness.', 'label': 6},
        {'text': 'I am happy about the success.', 'label': 6},
        {'text': 'I feel happy and cheerful.', 'label': 6},
        {'text': 'This creates happiness in me.', 'label': 6},

        # hopeful (12 samples)
        {'text': 'I am hopeful for the future.', 'label': 7},
        {'text': 'I feel hopeful about the outcome.', 'label': 7},
        {'text': 'This makes me feel hopeful.', 'label': 7},
        {'text': 'I am feeling hopeful today.', 'label': 7},
        {'text': 'I feel hopeful and optimistic.', 'label': 7},
        {'text': 'This gives me hope.', 'label': 7},
        {'text': 'I am hopeful about the changes.', 'label': 7},
        {'text': 'I feel hopeful and positive.', 'label': 7},
        {'text': 'This brings me hope.', 'label': 7},
        {'text': 'I am hopeful about the possibilities.', 'label': 7},
        {'text': 'I feel hopeful and confident.', 'label': 7},
        {'text': 'This creates hope in me.', 'label': 7},

        # overwhelmed (12 samples)
        {'text': 'I am feeling overwhelmed with tasks.', 'label': 8},
        {'text': 'I feel overwhelmed by the workload.', 'label': 8},
        {'text': 'This makes me feel overwhelmed.', 'label': 8},
        {'text': 'I am feeling overwhelmed today.', 'label': 8},
        {'text': 'I feel overwhelmed and stressed.', 'label': 8},
        {'text': 'This gives me overwhelm.', 'label': 8},
        {'text': 'I am overwhelmed by the situation.', 'label': 8},
        {'text': 'I feel overwhelmed and exhausted.', 'label': 8},
        {'text': 'This brings me overwhelm.', 'label': 8},
        {'text': 'I am overwhelmed by the pressure.', 'label': 8},
        {'text': 'I feel overwhelmed and drained.', 'label': 8},
        {'text': 'This creates overwhelm in me.', 'label': 8},

        # proud (12 samples)
        {'text': 'I am proud of my accomplishments.', 'label': 9},
        {'text': 'I feel proud of the results.', 'label': 9},
        {'text': 'This makes me feel proud.', 'label': 9},
        {'text': 'I am feeling proud today.', 'label': 9},
        {'text': 'I feel proud and accomplished.', 'label': 9},
        {'text': 'This gives me pride.', 'label': 9},
        {'text': 'I am proud of the achievement.', 'label': 9},
        {'text': 'I feel proud and satisfied.', 'label': 9},
        {'text': 'This brings me pride.', 'label': 9},
        {'text': 'I am proud of the success.', 'label': 9},
        {'text': 'I feel proud and confident.', 'label': 9},
        {'text': 'This creates pride in me.', 'label': 9},

        # sad (12 samples)
        {'text': 'I feel sad about the loss.', 'label': 10},
        {'text': 'I am sad about the situation.', 'label': 10},
        {'text': 'This makes me feel sad.', 'label': 10},
        {'text': 'I am feeling sad today.', 'label': 10},
        {'text': 'I feel sad and down.', 'label': 10},
        {'text': 'This gives me sadness.', 'label': 10},
        {'text': 'I am sad about the outcome.', 'label': 10},
        {'text': 'I feel sad and depressed.', 'label': 10},
        {'text': 'This brings me sadness.', 'label': 10},
        {'text': 'I am sad about the news.', 'label': 10},
        {'text': 'I feel sad and heartbroken.', 'label': 10},
        {'text': 'This creates sadness in me.', 'label': 10},

        # tired (12 samples)
        {'text': 'I am tired from working all day.', 'label': 11},
        {'text': 'I feel tired of the routine.', 'label': 11},
        {'text': 'This makes me feel tired.', 'label': 11},
        {'text': 'I am feeling tired today.', 'label': 11},
        {'text': 'I feel tired and exhausted.', 'label': 11},
        {'text': 'This gives me tiredness.', 'label': 11},
        {'text': 'I am tired of the situation.', 'label': 11},
        {'text': 'I feel tired and worn out.', 'label': 11},
        {'text': 'This brings me tiredness.', 'label': 11},
        {'text': 'I am tired of the stress.', 'label': 11},
        {'text': 'I feel tired and fatigued.', 'label': 11},
        {'text': 'This creates tiredness in me.', 'label': 11}
    ]

    print(f'✅ Recreated balanced dataset with {len(balanced_data)} samples')


# Check if tokenizer is defined, if not, recreate it
if 'tokenizer' not in globals():
    print('🔧 RECREATING TOKENIZER')
    print('=' * 40)
    tokenizer = AutoTokenizer.from_pretrained(specialized_model_name)
    print('✅ Tokenizer recreated successfully')


train_data, val_data = train_test_split(balanced_data, test_size=0.2, random_state=42, stratify=[d['label'] for d in balanced_data])

# Convert to datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


# Calculate class weights based on the training dataset
train_labels = [d['label'] for d in train_data]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Convert class weights to a dictionary mapping class index to weight
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Convert class weights dictionary to a tensor
class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float32)

# Move class weights tensor to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = class_weights_tensor.to(device)

print("✅ Calculated class weights:")
print(class_weights_dict)

# We need a custom trainer to pass class weights to the model's forward pass
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Initialize the weighted loss trainer
# Check if model, training_args, data_collator are defined, if not, recreate them
if 'model' not in globals():
    print('🔧 RECREATING MODEL')
    print('=' * 40)
    model = AutoModelForSequenceClassification.from_pretrained(
        specialized_model_name,
        num_labels=12,
        ignore_mismatched_sizes=True,
    )
    # Assuming 'emotions' is defined
    if 'emotions' in globals():
        model.config.id2label = {i: emotion for i, emotion in enumerate(emotions)}
        model.config.label2id = {emotion: i for i, emotion in enumerate(emotions)}
    else:
         print("⚠️ WARNING: 'emotions' list not defined, cannot set id2label/label2id in model config.")

    print('✅ Model recreated successfully')

if 'training_args' not in globals():
    print('⚙️  RECONFIGURING TRAINING ARGUMENTS')
    print('=' * 40)
    # Assuming best_run is defined and has hyperparameters
    if 'best_run' in globals() and hasattr(best_run, 'hyperparameters'):
        best_hyperparameters = best_run.hyperparameters
        training_args = TrainingArguments(
            output_dir='./corrected_emotion_model',
            learning_rate=best_hyperparameters.get('learning_rate', 2e-5), # Use default if not found
            per_device_train_batch_size=best_hyperparameters.get('per_device_train_batch_size', 16),
            per_device_eval_batch_size=best_hyperparameters.get('per_device_eval_batch_size', 16),
            num_train_epochs=best_hyperparameters.get('num_train_epochs', 5),
            weight_decay=best_hyperparameters.get('weight_decay', 0.01),
            logging_dir='./logs',
            logging_steps=10,
            eval_strategy="steps",
            save_strategy="steps",
            save_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            warmup_steps=100,
            dataloader_num_workers=0,
            save_total_limit=3
        )
        print('✅ Training arguments reconfigured with best_run hyperparameters')
    else:
        print("⚠️ WARNING: 'best_run' not defined or has no hyperparameters, using default TrainingArguments.")
        training_args = TrainingArguments(
            output_dir='./corrected_emotion_model',
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=5,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=10,
            eval_strategy="steps",
            save_strategy="steps",
            save_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            warmup_steps=100,
            dataloader_num_workers=0,
            save_total_limit=3
        )
        print('✅ Default Training arguments reconfigured')


if 'data_collator' not in globals():
    print('📦 RECREATING DATA COLLATOR')
    print('=' * 40)
    # Assuming tokenizer is defined
    if 'tokenizer' in globals():
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        print('✅ Data collator recreated successfully')
    else:
        print("⚠️ WARNING: 'tokenizer' not defined, cannot recreate data collator.")


# Check if compute_metrics is defined, if not, recreate it
if 'compute_metrics' not in globals():
    print('📊 RECREATING COMPUTE_METRICS FUNCTION')
    print('=' * 40)
    # Assuming emotions is defined
    if 'emotions' in globals():
        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.argmax(predictions, axis=1)

            # Calculate metrics
            report = classification_report(labels, predictions, target_names=emotions, output_dict=True)

            return {
                'f1': report['weighted avg']['f1-score'],
                'accuracy': report['accuracy'],
                'precision': report['weighted avg']['precision'],
                'recall': report['weighted avg']['recall']
            }
        print('✅ compute_metrics function recreated successfully')
    else:
         print("⚠️ WARNING: 'emotions' list not defined, cannot recreate compute_metrics function.")


final_trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print('✅ WeightedLossTrainer initialized successfully with class weights')

**Reasoning**:
The previous command failed because `compute_metrics` was not defined when initializing the `WeightedLossTrainer`. Although I included code to recreate it, the condition `'emotions' in globals()` was false, preventing its re-definition. I need to ensure that the `emotions` list is defined before attempting to recreate `compute_metrics`. I will add a check for `emotions` and recreate it if necessary, then ensure `compute_metrics` is properly defined before initializing the trainer.



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np
from transformers import Trainer, AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, DataCollatorWithPadding # Import all necessary transformers classes again
from datasets import Dataset # Re-import Dataset
from sklearn.metrics import classification_report # Import classification_report

# Check if specialized_model_name is defined, if not, define it
if 'specialized_model_name' not in globals():
    specialized_model_name = 'j-hartmann/emotion-english-distilroberta-base'
    print(f'🔧 Defined specialized_model_name: {specialized_model_name}')

# Check if emotions is defined, if not, define it
if 'emotions' not in globals():
    emotions = ['anxious', 'calm', 'content', 'excited', 'frustrated', 'grateful', 'happy', 'hopeful', 'overwhelmed', 'proud', 'sad', 'tired']
    print(f'🎯 Defined emotions list: {emotions}')


# Recreate train_data and val_data
# Check if balanced_data is defined, if not, recreate it
if 'balanced_data' not in globals():
     # Create balanced training dataset
    print('📊 RECREATING BALANCED DATASET')
    print('=' * 40)

    balanced_data = [
        # anxious (12 samples)
        {'text': 'I feel anxious about the presentation.', 'label': 0},
        {'text': 'I am anxious about the future.', 'label': 0},
        {'text': 'This makes me feel anxious.', 'label': 0},
        {'text': 'I am feeling anxious today.', 'label': 0},
        {'text': 'The uncertainty makes me anxious.', 'label': 0},
        {'text': 'I feel anxious about the results.', 'label': 0},
        {'text': 'This situation is making me anxious.', 'label': 0},
        {'text': 'I am anxious about the meeting.', 'label': 0},
        {'text': 'The pressure is making me anxious.', 'label': 0},
        {'text': 'I feel anxious about the decision.', 'label': 0},
        {'text': 'This is causing me anxiety.', 'label': 0},
        {'text': 'I am anxious about the changes.', 'label': 0},

        # calm (12 samples)
        {'text': 'I feel calm and peaceful.', 'label': 1},
        {'text': 'I am feeling calm today.', 'label': 1},
        {'text': 'This makes me feel calm.', 'label': 1},
        {'text': 'I am calm about the situation.', 'label': 1},
        {'text': 'I feel calm and relaxed.', 'label': 1},
        {'text': 'This gives me a sense of calm.', 'label': 1},
        {'text': 'I am feeling calm and centered.', 'label': 1},
        {'text': 'This brings me calm.', 'label': 1},
        {'text': 'I feel calm and at peace.', 'label': 1},
        {'text': 'I am calm about the outcome.', 'label': 1},
        {'text': 'This creates a feeling of calm.', 'label': 1},
        {'text': 'I feel calm and collected.', 'label': 1},

        # content (12 samples)
        {'text': 'I feel content with my life.', 'label': 2},
        {'text': 'I am content with the results.', 'label': 2},
        {'text': 'This makes me feel content.', 'label': 2},
        {'text': 'I am feeling content today.', 'label': 2},
        {'text': 'I feel content and satisfied.', 'label': 2},
        {'text': 'This gives me contentment.', 'label': 2},
        {'text': 'I am content with my choices.', 'label': 2},
        {'text': 'I feel content and fulfilled.', 'label': 2},
        {'text': 'This brings me contentment.', 'label': 2},
        {'text': 'I am content with the situation.', 'label': 2},
        {'text': 'I feel content and at ease.', 'label': 2},
        {'text': 'This creates contentment in me.', 'label': 2},

        # excited (12 samples)
        {'text': 'I am excited about the new opportunity.', 'label': 3},
        {'text': 'I feel excited about the future.', 'label': 3},
        {'text': 'This makes me feel excited.', 'label': 3},
        {'text': 'I am feeling excited today.', 'label': 3},
        {'text': 'I feel excited and enthusiastic.', 'label': 3},
        {'text': 'This gives me excitement.', 'label': 3},
        {'text': 'I am excited about the project.', 'label': 3},
        {'text': 'I feel excited and motivated.', 'label': 3},
        {'text': 'This brings me excitement.', 'label': 3},
        {'text': 'I am excited about the possibilities.', 'label': 3},
        {'text': 'I feel excited and energized.', 'label': 3},
        {'text': 'This creates excitement in me.', 'label': 3},

        # frustrated (12 samples)
        {'text': 'I am so frustrated with this project.', 'label': 4},
        {'text': 'I feel frustrated about the situation.', 'label': 4},
        {'text': 'This makes me feel frustrated.', 'label': 4},
        {'text': 'I am feeling frustrated today.', 'label': 4},
        {'text': 'I feel frustrated and annoyed.', 'label': 4},
        {'text': 'This gives me frustration.', 'label': 4},
        {'text': 'I am frustrated with the results.', 'label': 4},
        {'text': 'I feel frustrated and irritated.', 'label': 4},
        {'text': 'This brings me frustration.', 'label': 4},
        {'text': 'I am frustrated with the process.', 'label': 4},
        {'text': 'I feel frustrated and upset.', 'label': 4},
        {'text': 'This creates frustration in me.', 'label': 4},

        # grateful (12 samples)
        {'text': 'I am grateful for all the support.', 'label': 5},
        {'text': 'I feel grateful for the opportunity.', 'label': 5},
        {'text': 'This makes me feel grateful.', 'label': 5},
        {'text': 'I am feeling grateful today.', 'label': 5},
        {'text': 'I feel grateful and thankful.', 'label': 5},
        {'text': 'This gives me gratitude.', 'label': 5},
        {'text': 'I am grateful for the help.', 'label': 5},
        {'text': 'I feel grateful and appreciative.', 'label': 5},
        {'text': 'This brings me gratitude.', 'label': 5},
        {'text': 'I am grateful for the kindness.', 'label': 5},
        {'text': 'I feel grateful and blessed.', 'label': 5},
        {'text': 'This creates gratitude in me.', 'label': 5},

        # happy (12 samples)
        {'text': 'I am feeling really happy today!', 'label': 6},
        {'text': 'I feel happy about the news.', 'label': 6},
        {'text': 'This makes me feel happy.', 'label': 6},
        {'text': 'I am feeling happy today.', 'label': 6},
        {'text': 'I feel happy and joyful.', 'label': 6},
        {'text': 'This gives me happiness.', 'label': 6},
        {'text': 'I am happy with the results.', 'label': 6},
        {'text': 'I feel happy and delighted.', 'label': 6},
        {'text': 'This brings me happiness.', 'label': 6},
        {'text': 'I am happy about the success.', 'label': 6},
        {'text': 'I feel happy and cheerful.', 'label': 6},
        {'text': 'This creates happiness in me.', 'label': 6},

        # hopeful (12 samples)
        {'text': 'I am hopeful for the future.', 'label': 7},
        {'text': 'I feel hopeful about the outcome.', 'label': 7},
        {'text': 'This makes me feel hopeful.', 'label': 7},
        {'text': 'I am feeling hopeful today.', 'label': 7},
        {'text': 'I feel hopeful and optimistic.', 'label': 7},
        {'text': 'This gives me hope.', 'label': 7},
        {'text': 'I am hopeful about the changes.', 'label': 7},
        {'text': 'I feel hopeful and positive.', 'label': 7},
        {'text': 'This brings me hope.', 'label': 7},
        {'text': 'I am hopeful about the possibilities.', 'label': 7},
        {'text': 'I feel hopeful and confident.', 'label': 7},
        {'text': 'This creates hope in me.', 'label': 7},

        # overwhelmed (12 samples)
        {'text': 'I am feeling overwhelmed with tasks.', 'label': 8},
        {'text': 'I feel overwhelmed by the workload.', 'label': 8},
        {'text': 'This makes me feel overwhelmed.', 'label': 8},
        {'text': 'I am feeling overwhelmed today.', 'label': 8},
        {'text': 'I feel overwhelmed and stressed.', 'label': 8},
        {'text': 'This gives me overwhelm.', 'label': 8},
        {'text': 'I am overwhelmed by the situation.', 'label': 8},
        {'text': 'I feel overwhelmed and exhausted.', 'label': 8},
        {'text': 'This brings me overwhelm.', 'label': 8},
        {'text': 'I am overwhelmed by the pressure.', 'label': 8},
        {'text': 'I feel overwhelmed and drained.', 'label': 8},
        {'text': 'This creates overwhelm in me.', 'label': 8},

        # proud (12 samples)
        {'text': 'I am proud of my accomplishments.', 'label': 9},
        {'text': 'I feel proud of the results.', 'label': 9},
        {'text': 'This makes me feel proud.', 'label': 9},
        {'text': 'I am feeling proud today.', 'label': 9},
        {'text': 'I feel proud and accomplished.', 'label': 9},
        {'text': 'This gives me pride.', 'label': 9},
        {'text': 'I am proud of the achievement.', 'label': 9},
        {'text': 'I feel proud and satisfied.', 'label': 9},
        {'text': 'This brings me pride.', 'label': 9},
        {'text': 'I am proud of the success.', 'label': 9},
        {'text': 'I feel proud and confident.', 'label': 9},
        {'text': 'This creates pride in me.', 'label': 9},

        # sad (12 samples)
        {'text': 'I feel sad about the loss.', 'label': 10},
        {'text': 'I am sad about the situation.', 'label': 10},
        {'text': 'This makes me feel sad.', 'label': 10},
        {'text': 'I am feeling sad today.', 'label': 10},
        {'text': 'I feel sad and down.', 'label': 10},
        {'text': 'This gives me sadness.', 'label': 10},
        {'text': 'I am sad about the outcome.', 'label': 10},
        {'text': 'I feel sad and depressed.', 'label': 10},
        {'text': 'This brings me sadness.', 'label': 10},
        {'text': 'I am sad about the news.', 'label': 10},
        {'text': 'I feel sad and heartbroken.', 'label': 10},
        {'text': 'This creates sadness in me.', 'label': 10},

        # tired (12 samples)
        {'text': 'I am tired from working all day.', 'label': 11},
        {'text': 'I feel tired of the routine.', 'label': 11},
        {'text': 'This makes me feel tired.', 'label': 11},
        {'text': 'I am feeling tired today.', 'label': 11},
        {'text': 'I feel tired and exhausted.', 'label': 11},
        {'text': 'This gives me tiredness.', 'label': 11},
        {'text': 'I am tired of the situation.', 'label': 11},
        {'text': 'I feel tired and worn out.', 'label': 11},
        {'text': 'This brings me tiredness.', 'label': 11},
        {'text': 'I am tired of the stress.', 'label': 11},
        {'text': 'I feel tired and fatigued.', 'label': 11},
        {'text': 'This creates tiredness in me.', 'label': 11}
    ]

    print(f'✅ Recreated balanced dataset with {len(balanced_data)} samples')


# Check if tokenizer is defined, if not, recreate it
if 'tokenizer' not in globals():
    print('🔧 RECREATING TOKENIZER')
    print('=' * 40)
    tokenizer = AutoTokenizer.from_pretrained(specialized_model_name)
    print('✅ Tokenizer recreated successfully')


train_data, val_data = train_test_split(balanced_data, test_size=0.2, random_state=42, stratify=[d['label'] for d in balanced_data])

# Convert to datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


# Calculate class weights based on the training dataset
train_labels = [d['label'] for d in train_data]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Convert class weights to a dictionary mapping class index to weight
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Convert class weights dictionary to a tensor
class_weights_tensor = torch.tensor(list(class_weights_dict.values()), dtype=torch.float32)

# Move class weights tensor to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights_tensor = class_weights_tensor.to(device)

print("✅ Calculated class weights:")
print(class_weights_dict)

# Define compute_metrics function if not defined
if 'compute_metrics' not in globals():
    print('📊 DEFINING COMPUTE_METRICS FUNCTION')
    print('=' * 40)
    # Assuming emotions is defined now
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        # Calculate metrics
        # Check if emotions is defined before using it
        target_names = emotions if 'emotions' in globals() else None
        report = classification_report(labels, predictions, target_names=target_names, output_dict=True)

        return {
            'f1': report['weighted avg']['f1-score'],
            'accuracy': report['accuracy'],
            'precision': report['weighted avg']['precision'],
            'recall': report['weighted avg']['recall']
        }
    print('✅ compute_metrics function defined')


# We need a custom trainer to pass class weights to the model's forward pass
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Initialize the weighted loss trainer
# Check if model, training_args, data_collator are defined, if not, recreate them
if 'model' not in globals():
    print('🔧 RECREATING MODEL')
    print('=' * 40)
    model = AutoModelForSequenceClassification.from_pretrained(
        specialized_model_name,
        num_labels=12,
        ignore_mismatched_sizes=True,
    )
    # Assuming 'emotions' is defined
    if 'emotions' in globals():
        model.config.id2label = {i: emotion for i, emotion in enumerate(emotions)}
        model.config.label2id = {emotion: i for i, emotion in enumerate(emotions)}
    else:
         print("⚠️ WARNING: 'emotions' list not defined, cannot set id2label/label2id in model config.")

    print('✅ Model recreated successfully')

if 'training_args' not in globals():
    print('⚙️  RECONFIGURING TRAINING ARGUMENTS')
    print('=' * 40)
    # Assuming best_run is defined and has hyperparameters
    if 'best_run' in globals() and hasattr(best_run, 'hyperparameters'):
        best_hyperparameters = best_run.hyperparameters
        training_args = TrainingArguments(
            output_dir='./corrected_emotion_model',
            learning_rate=best_hyperparameters.get('learning_rate', 2e-5), # Use default if not found
            per_device_train_batch_size=best_hyperparameters.get('per_device_train_batch_size', 16),
            per_device_eval_batch_size=best_hyperparameters.get('per_device_eval_batch_size', 16),
            num_train_epochs=best_hyperparameters.get('num_train_epochs', 5),
            weight_decay=best_hyperparameters.get('weight_decay', 0.01),
            logging_dir='./logs',
            logging_steps=10,
            eval_strategy="steps",
            save_strategy="steps",
            save_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            warmup_steps=100,
            dataloader_num_workers=0,
            save_total_limit=3
        )
        print('✅ Training arguments reconfigured with best_run hyperparameters')
    else:
        print("⚠️ WARNING: 'best_run' not defined or has no hyperparameters, using default TrainingArguments.")
        training_args = TrainingArguments(
            output_dir='./corrected_emotion_model',
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=5,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=10,
            eval_strategy="steps",
            save_strategy="steps",
            save_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            warmup_steps=100,
            dataloader_num_workers=0,
            save_total_limit=3
        )
        print('✅ Default Training arguments reconfigured')


if 'data_collator' not in globals():
    print('📦 RECREATING DATA COLLATOR')
    print('=' * 40)
    # Assuming tokenizer is defined
    if 'tokenizer' in globals():
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        print('✅ Data collator recreated successfully')
    else:
        print("⚠️ WARNING: 'tokenizer' not defined, cannot recreate data collator.")


final_trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print('✅ WeightedLossTrainer initialized successfully with class weights')

**Reasoning**:
The `WeightedLossTrainer` has been successfully initialized with the calculated class weights. The next step according to the subtask is to train the model using this modified training process to incorporate the chosen technique (class weighting). I will now train the final model using the `final_trainer` instance.



In [ ]:
# Train the final model with weighted loss
print("🚀 STARTING FINAL MODEL TRAINING WITH CLASS WEIGHTS")
print('=' * 40)
final_trainer.train()
print("✅ Final model training completed successfully with class weights")

**Reasoning**:
The previous command failed because the `compute_loss` method in my custom `WeightedLossTrainer` does not accept the `num_items_in_batch` argument, which is now being passed by the updated `Trainer` class in the `transformers` library. I need to modify the `compute_loss` method signature in `WeightedLossTrainer` to accept this argument, even if I don't use it. I will redefine the `WeightedLossTrainer` with the updated `compute_loss` signature and then re-initialize and train the model.



In [ ]:
# Redefine the WeightedLossTrainer with the updated compute_loss signature
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor.to(logits.device)) # Ensure weights are on the same device as logits
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Initialize the weighted loss trainer again with the corrected class
final_trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Train the final model with weighted loss
print("🚀 STARTING FINAL MODEL TRAINING WITH CLASS WEIGHTS (Corrected Trainer)")
print('=' * 40)
final_trainer.train()
print("✅ Final model training completed successfully with class weights")

## Retrain and evaluate

### Subtask:
Retrain and evaluate the model with the chosen technique (class weighting).


**Reasoning**:
I need to evaluate the performance of the final model after training with class weights on the validation set. This will show the impact of the class weighting technique. I will use the `evaluate` method of the `final_trainer` and print the relevant metrics.



In [ ]:
# Evaluate the final model with class weights
print('📊 EVALUATING FINAL MODEL WITH CLASS WEIGHTS')
print('=' * 40)

final_results_weighted = final_trainer.evaluate()
print(f'Final Weighted F1 Score: {final_results_weighted["eval_f1"]:.3f}')
print(f'Final Weighted Accuracy: {final_results_weighted["eval_accuracy"]:.3f}')
print(f'Final Weighted Precision: {final_results_weighted["eval_precision"]:.3f}')
print(f'Final Weighted Recall: {final_results_weighted["eval_recall"]:.3f}')

## Summary:

### Data Analysis Key Findings

*   The initial attempt to implement class weighting faced issues due to undefined variables and missing imports, which were resolved by recreating necessary data and ensuring all dependencies were imported.
*   A `SyntaxError` in the manually created balanced dataset was identified and corrected.
*   The custom `WeightedLossTrainer` initially failed due to an incorrect method signature for `compute_loss`; this was fixed by adding the `num_items_in_batch` parameter and ensuring class weights were on the correct device.
*   After resolving these issues, the model was successfully trained using the `WeightedLossTrainer` with calculated class weights.
*   Evaluation of the model trained with class weights on the validation set yielded the following metrics: F1 Score: 0.396, Accuracy: 0.483, Precision: 0.405, and Recall: 0.483.
*   `UndefinedMetricWarning` messages were observed during evaluation, indicating that the model did not predict any samples for certain emotion classes in the validation set.

### Insights or Next Steps

*   The observed `UndefinedMetricWarning` suggests that even with class weighting, the model may still struggle to correctly classify all emotion categories, particularly those that might still be under-represented or have subtle distinctions in the data. Further analysis into the specific classes causing the warnings is needed.
*   Consider exploring data augmentation techniques for the under-represented classes in conjunction with or as an alternative to class weighting to provide more diverse training examples and potentially improve performance on these classes.
